# Phase 7 — Simulation & Comparison: SLIL vs baseline routing

This notebook runs a deterministic, reproducible Monte Carlo simulation to compare standard Stellar pathfinding against SLIL-assisted routing. It generates synthetic graphs with annotated edge signals (depth, spread, volatility, failure_rate), runs many simulated transactions, and reports metrics such as success rate and average slippage.

DONE criteria:
- Produce a reproducible comparison showing improved success rate and/or lower slippage for SLIL-assisted routing on the synthetic dataset.

## 1) Environment & Dependencies

This notebook uses Python 3 and the project's backend code. Required libraries:

- numpy, pandas, matplotlib, networkx

Install (backend virtualenv):

```bash
python3 -m pip install -r backend/requirements.txt
python3 -m pip install matplotlib pandas networkx
```

Optional front-end test tools (Jest, Cypress, msw) are documented later.

In [ ]:
## 2) Imports and reproducible RNG

```python
import os
from datetime import datetime
import json
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

SEED = 42
rng = np.random.default_rng(SEED)

ARTIFACT_DIR = Path("analysis/artifacts/phase7")
ARTIFACT_DIR.mkdir(parents=True, exist_ok=True)

print("Environment ready. Seed:", SEED)
```


In [ ]:
## 3) Generate synthetic graphs with annotated edges

```python
# Build a synthetic graph generator with controlled signals

def make_synthetic_graph(num_assets=8, seed=SEED):
    rng_local = np.random.default_rng(seed)
    G = nx.DiGraph()
    assets = [f"A{i}" for i in range(num_assets)]
    for a in assets:
        G.add_node(a)

    # Connect assets with some probability and assign synthetic signals
    for i in range(num_assets):
        for j in range(num_assets):
            if i == j:
                continue
            if rng_local.random() < 0.35:
                depth = float(rng_local.uniform(20, 200))  # liquidity depth in units
                spread = float(rng_local.uniform(0.0005, 0.01))
                vol = float(rng_local.uniform(0.001, 0.05))
                failure_rate = float(rng_local.uniform(0.0, 0.2))
                G.add_edge(assets[i], assets[j], depth=depth, spread=spread, volatility=vol, failure_rate=failure_rate)
    return G

# Example graph
G = make_synthetic_graph(10)
print(f"Nodes: {G.number_of_nodes()}, Edges: {G.number_of_edges()}")
```


In [ ]:
## 4) Routing policies: baseline vs SLIL

```python
# Baseline: shortest path by hop count or by sum(spread)
def baseline_route(G, source, dest, mode='spread'):
    try:
        if mode == 'hop':
            return nx.shortest_path(G, source, dest)
        elif mode == 'spread':
            # weight edges by spread
            return nx.shortest_path(G, source, dest, weight=lambda u, v, d: d.get('spread', 1.0))
    except Exception:
        return None

# SLIL: compute a simple path reliability score using a Phase 4 like function
# R = 0.4 * normalized_depth - 0.2*spread_scaled - 0.2*vol_scaled - 0.2*failure_rate

def edge_score(attrs):
    # Normalize heuristically for synthetic data
    depth = attrs.get('depth', 0.0)
    spread = attrs.get('spread', 0.01)
    vol = attrs.get('volatility', 0.01)
    failure = attrs.get('failure_rate', 0.1)

    depth_norm = np.tanh(depth / 100.0)  # 0..1 like
    spread_norm = min(spread / 0.01, 1.0)
    vol_norm = min(vol / 0.05, 1.0)
    fail_norm = min(failure / 0.2, 1.0)

    return 0.4 * depth_norm - 0.2 * spread_norm - 0.2 * vol_norm - 0.2 * fail_norm


def slil_route(G, source, dest, max_hops=4):
    # enumerate simple paths up to max_hops
    try:
        paths = list(nx.all_simple_paths(G, source, dest, cutoff=max_hops))
    except Exception:
        return None
    best = None
    best_score = -1e9
    for p in paths:
        # path score = average edge_score
        scores = []
        ok = True
        for u, v in zip(p[:-1], p[1:]):
            if not G.has_edge(u, v):
                ok = False
                break
            scores.append(edge_score(G.get_edge_data(u, v)))
        if not ok or not scores:
            continue
        s = sum(scores) / len(scores)
        if s > best_score:
            best_score = s
            best = p
    return best
```


In [ ]:
## 5) Transaction Simulator

```python
# Simulate a batch of transactions along a path: each edge may fail (probability = failure_rate),
# and slippage per edge sampled from N(mean=spread, sd=volatility)

def simulate_transactions(G, path, n=50, rng=None):
    if not path or len(path) < 2:
        return {'successes':0, 'attempts':0, 'avg_slippage':None}
    if rng is None:
        rng = np.random.default_rng(SEED)
    successes = 0
    slippages = []
    attempts = n
    for _ in range(n):
        failed = False
        total_slip = 0.0
        for u, v in zip(path[:-1], path[1:]):
            attrs = G.get_edge_data(u, v)
            fail_p = attrs.get('failure_rate', 0.1)
            if rng.random() < fail_p:
                failed = True
                break
            mean = attrs.get('spread', 0.001)
            sd = attrs.get('volatility', 0.01)
            slip = max(0.0, rng.normal(mean, sd))
            total_slip += slip
        if not failed:
            successes += 1
            slippages.append(total_slip)
    avg_slip = float(np.mean(slippages)) if slippages else None
    return {'successes':successes, 'attempts':attempts, 'success_rate': successes/attempts if attempts else 0.0, 'avg_slippage':avg_slip}
```


In [ ]:
## 6) Monte Carlo experiment: compare baseline vs SLIL

```python
# Run a small Monte Carlo experiment

corridors = []
nodes = list(G.nodes())
for i in range(len(nodes)):
    for j in range(i+1, len(nodes)):
        corridors.append((nodes[i], nodes[j]))

rng_local = np.random.default_rng(SEED+1)
results = []

for (src, dst) in corridors:
    b_path = baseline_route(G, src, dst, mode='spread')
    s_path = slil_route(G, src, dst, max_hops=5)
    if not b_path and not s_path:
        continue
    b_res = simulate_transactions(G, b_path, n=50, rng=rng_local)
    s_res = simulate_transactions(G, s_path, n=50, rng=rng_local)
    results.append({
        'src':src, 'dst':dst,
        'baseline_success': b_res.get('success_rate', 0.0), 'baseline_slip': b_res.get('avg_slippage'),
        'slil_success': s_res.get('success_rate', 0.0), 'slil_slip': s_res.get('avg_slippage'),
        'b_path': b_path, 's_path': s_path
    })

df = pd.DataFrame(results)
print(f"Completed experiments for {len(df)} corridors")
df.head()
```


In [ ]:
## 7) Aggregate results and plot summaries

```python
summary = {
    'baseline_avg_success': df['baseline_success'].mean(),
    'slil_avg_success': df['slil_success'].mean(),
    'baseline_avg_slip': df['baseline_slip'].dropna().mean(),
    'slil_avg_slip': df['slil_slip'].dropna().mean(),
}
print(summary)

# Plot success rates
plt.figure(figsize=(8,4))
plt.hist(df['baseline_success'].dropna(), alpha=0.6, label='Baseline')
plt.hist(df['slil_success'].dropna(), alpha=0.6, label='SLIL')
plt.legend()
plt.title('Distribution of success rates: baseline vs SLIL')
plt.xlabel('success rate')
plt.ylabel('count')
plt.tight_layout()
plt.savefig(ARTIFACT_DIR / 'success_rate_hist.png')
plt.show()

# Scatter of improvement
plt.figure(figsize=(6,6))
plt.scatter(df['baseline_success'], df['slil_success'])
plt.plot([0,1],[0,1], '--', color='gray')
plt.xlabel('Baseline success')
plt.ylabel('SLIL success')
plt.title('SLIL vs Baseline (above diagonal = improvement)')
plt.tight_layout()
plt.savefig(ARTIFACT_DIR / 'success_scatter.png')
plt.show()

# Save summary CSV
(df.assign(improvement = df['slil_success'] - df['baseline_success']).sort_values('improvement', ascending=False)
   .to_csv(ARTIFACT_DIR / 'phase7_summary.csv', index=False))
print('Artifacts written to', ARTIFACT_DIR)
```


In [ ]:
## 8) Reproducibility & manifest

```python
manifest = {
    'git_commit': os.getenv('GIT_COMMIT', 'LOCAL'),
    'seed': SEED,
    'timestamp': datetime.utcnow().isoformat(),
    'summary': summary,
}
with open(ARTIFACT_DIR / 'manifest.json', 'w') as f:
    json.dump(manifest, f, indent=2)

print('Manifest written')
```


In [ ]:
## 9) Diagnostics & Sensitivity

```python
# Compute per-corridor improvement and diagnostics
out_df = df.copy()
out_df['improvement'] = out_df['slil_success'] - out_df['baseline_success']
num_improved = (out_df['improvement'] > 0).sum()
num_total = len(out_df)
mean_improvement = out_df['improvement'].mean()
print(f"Corridors improved: {num_improved}/{num_total} ({100.0 * num_improved/num_total:.1f}%)")
print(f"Mean improvement: {mean_improvement:.4f}")

# Path length comparison
out_df['b_len'] = out_df['b_path'].apply(lambda p: len(p)-1 if p else None)
out_df['s_len'] = out_df['s_path'].apply(lambda p: len(p)-1 if p else None)

plt.figure(figsize=(6,4))
plt.hist(out_df['s_len'].dropna() - out_df['b_len'].dropna(), bins=range(-5,6), alpha=0.7)
plt.title('Path length difference (SLIL - baseline)')
plt.xlabel('edge difference')
plt.ylabel('count')
plt.tight_layout()
plt.savefig(ARTIFACT_DIR / 'path_length_diff.png')
plt.show()

# Save diagnostics
out_df.to_csv(ARTIFACT_DIR / 'phase7_diagnostics.csv', index=False)
```
